# Capítulo 08: Manejo avanzado del dinero

💰 Únete a la comunidad de [Discord](https://discord.gg/z3dx5XpkX4)

📚 Puedes leer nuestro libro en [Amazon](https://www.amazon.es/Python-para-finanzas-trading-algor%C3%ADtmico-ebook/dp/B0BT4ZS9Q3)

🖥️ El canal de [YouTube de Quantreo's](https://www.youtube.com/channel/UCp7jckfiEglNf_Gj62VR0pw) (en inglés) y el de [Frogames](https://www.youtube.com/channel/UCMUxXNYrVCv6-bQakhomvBg) en Español

In [11]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
mt5.initialize()

True

### Extraer las posiciones actuales

In [4]:
mt5.positions_get()

(TradePosition(ticket=231598598, time=1676299226, time_msc=1676299226671, time_update=1676299226, time_update_msc=1676299226671, type=0, magic=0, identifier=231598598, reason=3, volume=1.0, price_open=1.06976, sl=0.0, tp=0.0, price_current=1.06979, swap=0.0, profit=0.28, symbol='EURUSD', comment='', external_id='1049AE94DC6A6BA1'),)

In [12]:
def resume():
    """ Devuelve las posiciones actuales. position=0 --> Buy """    
    # Definimos el nombre de las columnas que vamos a calcular
    columns = ["ticket", "position", "symbol", "volume", "magic", "profit", "price", "tp", "sl","trade_size"]

    # Obtenemos la lista de posiciones abiertas
    list_current = mt5.positions_get()

    # Creamos un dataframe vacío
    summary = pd.DataFrame()

    # Bucle para añadir cada fila al dataframe
    for element in list_current:
        element_pandas = pd.DataFrame([element.ticket, element.type, element.symbol, element.volume, element.magic,
                                       element.profit, element.price_open, element.tp,
                                       element.sl, mt5.symbol_info(element.symbol).trade_contract_size],
                                      index=columns).transpose()
        summary = pd.concat((summary, element_pandas), axis=0)
    
    try:
        summary["profit %"] = summary.profit / (summary.price * summary.trade_size * summary.volume)
        summary = summary.reset_index(drop=True)
    except:
        pass
    return summary

In [13]:
resume()

,ticket,position,symbol,volume,magic,profit,price,tp,sl,trade_size,profit %
0,231635677,0,EURUSD,0.1,0,0.21,1.07429,0.0,0.0,10000.0,0.000195


### TRAILING STOP LOSS

In [14]:
# Crear un diccionario que contenga el precio máximo y mínimo de cada símbolo 
max_price = dict()
min_price = dict()

# Bucle infinito
while True:
    
    # Extraems las posiciones actuales
    summary = resume()
    
    # Verificación: ¿Hay alguna posición abierta?
    if summary.shape[0] >0:
        for i in range(summary.shape[0]):
            
            # Extraemos la información
            row = summary.iloc[i]
            symbol = row["symbol"]
            
            # Añadir la clave si no está en la lista de claves
            if symbol not in max_price.keys():
                max_price[symbol]=row["price"]
                
            if symbol not in min_price.keys():
                min_price[symbol]=row["price"]
                
                  
        

            """ CASO 1: Cambiar dinámicamente el stop loss para una BUY ORDER """
            # Trailing stop loss para una orden de compra
            if row["position"] == 0:
               
                # Extraemos el precio actual 
                current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2
                
                # Calcula la distancia entre el precio actual y el precio máximo
                from_sl_to_curent_price = current_price - row["sl"]
                from_sl_to_max_price = max_price[symbol] - row["sl"]
                
                
                #Si el precio actual es mayor que el precio máximo anterior --> nuevo precio máximo
                if current_price > max_price[symbol]:
                    max_price[symbol] = current_price
                    
                
                # Hallar la diferencia entre el actual menos el máximo. 
                if from_sl_to_curent_price > from_sl_to_max_price:
                    difference = from_sl_to_curent_price - from_sl_to_max_price

                    # Elegimos el filling mode
                    filling_type = mt5.symbol_info(symbol).filling_mode

                    # Elegimos el punto
                    point = mt5.symbol_info(symbol).point

                    # Cambiamos el sl
                    request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": row["ticket"],
                    "volume": row["volume"],
                    "type": mt5.ORDER_TYPE_BUY,
                    "price": row["price"],
                    "sl": row["sl"] + difference,
                    "type_filling": filling_type,
                    "type_time": mt5.ORDER_TIME_GTC,
                    }
                    
                    information = mt5.order_send(request)
                    print(information)
                    
            
            """ CASO 2:  Cambiar dinámicamente el stop loss para una SELL ORDER """
            # Trailing stop loss para una orden de venta
            if row["position"] == 1:
                
               
                # Extraemos el precio actual 
                current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2
                
                # Calcula la distancia entre el precio actual y el precio máximo  
                from_sl_to_curent_price = row["sl"] - current_price
                from_sl_to_min_price = row["sl"] - min_price[symbol]
                
                 # Si el precio actual es menor que el precio mínimo anterior --> nuevo precio máximo
                if current_price < min_price[symbol]:
                    min_price[symbol] = current_price
                                
                    
                # Hallar la diferencia entre el actual menos el máximo. 
                if from_sl_to_curent_price > from_sl_to_min_price:
                    difference = from_sl_to_curent_price - from_sl_to_min_price 

                    # Elegimos el filling mode
                    filling_type = mt5.symbol_info(symbol).filling_mode

                    # Elegimos el punto
                    point = mt5.symbol_info(symbol).point

                    # Cambiamos el sl
                    request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": row["ticket"],
                    "volume": row["volume"],
                    "type": mt5.ORDER_TYPE_SELL,
                    "price": row["price"],
                    "sl": row["sl"] - difference,
                    "type_filling": filling_type,
                    "type_time": mt5.ORDER_TIME_GTC,
                    }
                    
                
                    information = mt5.order_send(request)
                    print(information)
    print(max_price)
    # Evitamos el ruido
    time.sleep(1)
    
    

OrderSendResult(retcode=10013, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid request', request_id=0, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='EURUSD', volume=0.1, price=1.07429, stoplimit=0.0, sl=0.00021500000000007624, tp=0.0, deviation=0, type=0, type_filling=3, type_time=0, expiration=0, comment='', position=231635677, position_by=0))
{'EURUSD': 1.074505}
{'EURUSD': 1.074505}
OrderSendResult(retcode=10013, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid request', request_id=0, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='EURUSD', volume=0.1, price=1.07429, stoplimit=0.0, sl=5.000000000032756e-06, tp=0.0, deviation=0, type=0, type_filling=3, type_time=0, expiration=0, comment='', position=231635677, position_by=0))
{'EURUSD': 1.07451}
{'EURUSD': 1.07451}
{'EURUSD': 1.07451}
OrderSendResult(retcode=10013, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, 

KeyboardInterrupt: 

In [18]:
def trailing_stop_loss():
    global max_price, min_price, summary
    
    # Extraer las posiciones abiertas actuales
    summary = resume()
    
    # Verification: Is there any open position?
    if summary.shape[0] >0:
        for i in range(summary.shape[0]):
            
            # Extract information
            row = summary.iloc[i]
            symbol = row["symbol"]
            
        
                         
        

            """ CASE 1: Change dynamicly the stop loss for a BUY ORDER """
            # Trailing stop loss for a buy order
            if row["position"] == 0:
               
                if symbol not in max_price.keys():
                    max_price[symbol]=row["price"]
                
                # Extract current price 
                current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2
                
                #Compute distance between current price an max price
                from_sl_to_curent_price = current_price - row["sl"]
                from_sl_to_max_price = max_price[symbol] - row["sl"]
                
                
                # If current price is greater than preivous max price --> new max price
                if current_price > max_price[symbol]:
                    max_price[symbol] = current_price
                    
                
                # Find the difference between the current minus max 
                if from_sl_to_curent_price > from_sl_to_max_price:
                    difference = from_sl_to_curent_price - from_sl_to_max_price

                    # Set filling mode
                    filling_type = mt5.symbol_info(symbol).filling_mode

                    # Set the point
                    point = mt5.symbol_info(symbol).point

                    # Change the sl
                    request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": row["ticket"],
                    "volume": row["volume"],
                    "type": mt5.ORDER_TYPE_BUY,
                    "price": row["price"],
                    "sl": row["sl"] + difference,
                    "type_filling": filling_type,
                    "type_time": mt5.ORDER_TIME_GTC,
                    }
                    
                    information = mt5.order_send(request)
                    print(information)
                    
            
            """ CASE 2: Change dynamicly the stop loss for a SELL ORDER """
            # Trailing stop loss for a sell order
            if row["position"] == 1:
                
                if symbol not in min_price.keys():
                    min_price[symbol]=row["price"]
                    
                # Extract current price 
                current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2
                
                
                
                #Compute distance between current price an max price
                from_sl_to_curent_price = row["sl"] - current_price
                from_sl_to_min_price = row["sl"] - min_price[symbol]
                
                 # If current price is greater than preivous max price --> new max price
                if current_price < min_price[symbol]:
                    min_price[symbol] = current_price
                                
                    
                # Find the difference between the current minus max 
                if from_sl_to_curent_price > from_sl_to_min_price:
                    difference = from_sl_to_curent_price - from_sl_to_min_price 

                    # Set filling mode
                    filling_type = 1#mt5.symbol_info(symbol).filling_mode

                    # Set the point
                    point = mt5.symbol_info(symbol).point

                    # Change the sl
                    request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": row["ticket"],
                    "volume": row["volume"],
                    "type": mt5.ORDER_TYPE_SELL,
                    "price": row["price"],
                    "sl": row["sl"] - difference,
                    "type_filling": filling_type,
                    "type_time": mt5.ORDER_TIME_GTC,
                    }
                    
                
                    information = mt5.order_send(request)
                    print(information)

In [16]:
def verif_tsl():
    global max_price, min_price, summary
    
    #print("MAX", max_price)
    
    #print("MIN", min_price)
    
    if len(summary)>0:
        buy_open_positions = summary.loc[summary["position"]==0]["symbol"]
        sell_open_positions = summary.loc[summary["position"]==1]["symbol"]
    else:
        buy_open_positions = []
        sell_open_positions = []
    
    """ SI CERRAMOS UNA DE LAS POSICIONES DEBEMOS BORRAR EL PRECIO EN LOS DICCIONARIOS DE PRECIOS MÁXIMO Y MÍNIMO"""
    if len(max_price) != len(buy_open_positions) and len(buy_open_positions) >0:
        symbol_to_delete = []
        
        for symbol in max_price.keys():

            if symbol not in list(buy_open_positions):
                symbol_to_delete.append(symbol)
        
        for symbol in symbol_to_delete:
            del max_price[symbol]
            
    if len(min_price) != len(sell_open_positions) and len(sell_open_positions) >0:
        symbol_to_delete = []
        
        for symbol in min_price.keys():

            if symbol not in list(sell_open_positions):
                symbol_to_delete.append(symbol)
        
        for symbol in symbol_to_delete:
            del min_price[symbol]
    
    if len(buy_open_positions) == 0:
        max_price={}
        
    if len(sell_open_positions) == 0:
        min_price={}
    

In [21]:
# Create a dictionnary to contain the maxiumum price of each symbol 
max_price = dict()
min_price = dict()

# Infinite loop
while True:
    trailing_stop_loss()

    
    verif_tsl()
    
    
    time.sleep(1)

OrderSendResult(retcode=10013, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid request', request_id=0, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='USDJPY', volume=0.1, price=131.99, stoplimit=0.0, sl=0.0024999999999977263, tp=0.0, deviation=0, type=0, type_filling=3, type_time=0, expiration=0, comment='', position=231636149, position_by=0))
OrderSendResult(retcode=10013, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid request', request_id=0, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='USDJPY', volume=0.1, price=131.99, stoplimit=0.0, sl=0.0020000000000095497, tp=0.0, deviation=0, type=0, type_filling=3, type_time=0, expiration=0, comment='', position=231636149, position_by=0))
OrderSendResult(retcode=10013, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid request', request_id=0, retcode_external=0, request=TradeRequest(action=6, magic=0, orde

KeyboardInterrupt: 

### Portfolio o Basket TP/SL

In [22]:
def verification_basket_sl(sl):
    """Necesitamos poner el signo de sl:
    Stop loss -1%: sl=-1
    """
    list_current_position = list(mt5.positions_get())
    sum_price = sum_profit = 0
    for position in list_current_position:
        symbol = position.symbol
        trade_contract_size = mt5.symbol_info(symbol).trade_contract_size
        sum_price += (position.volume * position.price_open * trade_contract_size)
        sum_profit += position.profit
    profit_in_pct = 100 * sum_profit / sum_price
    condition = profit_in_pct < sl
    print(profit_in_pct)
    return condition

In [25]:
verification_basket_sl(-1)

-0.0012500231143327354


False

In [26]:
def verification_basket_tp(tp):
    list_current_position = list(mt5.positions_get())
    sum_price = sum_profit = 0
    for position in list_current_position:
        symbol = position.symbol
        trade_contract_size = mt5.symbol_info(symbol).trade_contract_size
        sum_price += (position.volume * position.price_open * trade_contract_size)
        sum_profit += position.profit
    profit_in_pct = 100 * sum_profit / sum_price
    condition = profit_in_pct > tp
    print(profit_in_pct)
    return condition

In [29]:
verification_basket_tp(1)

-0.0016346456110504998


False